# U-Net

The first approach would be training a U-Net model and use the Encoder as a classifier.

In [ ]:
! pip install torch
! pip install torchvision
! pip install pandas
! pip install tqdm
! pip install matplotlib
! pip install psutil
! pip install segmentation-models-pytorch
! pip install wandb

In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tim-mach (idp2024). Use `wandb login --relogin` to force relogin


True

# Prepare Datasets

In [1]:
from dataset import SegmentationDataset
from torch.utils.data import DataLoader, random_split
import os
from torchvision.transforms import Compose, ToTensor, Grayscale, Normalize


train_image_path = './FIVES/train/Original'
train_label_path = './FIVES/train/GroundTruth'
test_image_path = './FIVES/test/Original'
test_label_path = './FIVES/test/GroundTruth'

# Define transformations for images
image_transform = Compose([
    Grayscale(num_output_channels=1),  # Convert image to grayscale
    ToTensor()                         # Convert the image to a PyTorch tensor
])

# Define transformations for labels, if needed
label_transform = Compose([
    ToTensor()           # Convert label to a tensor
])

dataset = SegmentationDataset(train_image_path, train_label_path, image_transform, label_transform)
testset = SegmentationDataset(test_image_path, test_label_path, image_transform, label_transform)

# Prepare DataLoader
train_size, val_size = int(0.9 * len(dataset)), int(0.1 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(f'Number of samples in the training set: {len(train_dataset)}, validation set: {len(val_dataset)}')
print(f'Number of samples in the test set: {len(testset)}')

trainloader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=8)
validationloader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=8)
testloader = DataLoader(testset, batch_size=2, shuffle=False, num_workers=8)

Number of samples in the training set: 540, validation set: 60
Number of samples in the test set: 200


# Visualize Dataset

In [ ]:

import matplotlib.pyplot as plt

def show_images(images, labels, num_images=5):
    fig, axs = plt.subplots(nrows=num_images, ncols=2, figsize=(10, num_images * 5))
    for i in range(num_images):
        img = images[i].numpy().transpose((1, 2, 0))  # Convert from PyTorch tensor format and channel first to channel last
        lbl = labels[i].numpy().squeeze()            # Remove extra dimensions
        
        if num_images == 1:
            ax_img = axs[0]
            ax_lbl = axs[1]
        else:
            ax_img = axs[i, 0]
            ax_lbl = axs[i, 1]
        
        ax_img.imshow(img, cmap='gray')
        ax_img.set_title('Retina')
        ax_img.axis('off')
        
        ax_lbl.imshow(lbl, cmap='gray')
        ax_lbl.set_title('Segmentation Map')
        ax_lbl.axis('off')

    plt.tight_layout()
    plt.show()

show_images(*zip(*[dataset[i] for i in range(5)]))

# Training

In [2]:
import segmentation_models_pytorch as smp
import torch.nn as nn
import torch
from util import model_pipeline
import matplotlib.pyplot as plt

config = {
    'model': 'Unet_resnext50_32x4d_with_DiceLoss',
    'learnnig_rate': 0.001,
    'epochs': 10,
    'batch_size': 2,
    'optimizer': 'Adam',
    'criterion': 'DiceLoss',
}

model = smp.Unet('resnext50_32x4d', in_channels=1, classes=1)
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=config['learnnig_rate'])
criterion = smp.losses.DiceLoss(mode='binary')

model, train_losses, val_losses = model_pipeline(model, trainloader=trainloader, validationloader=validationloader, testloader=testloader, criterion=criterion, optimizer=optimizer,project='retina-segmentation',model_name=config['model'],device=device, config=config)

# Plotting the training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()

/home/tim_ivan/idp/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tim-mach (idp2024). Use `wandb login --relogin` to force relogin


Epoch 1, Batch 10, Loss: 0.7835
Epoch 1, Batch 20, Loss: 0.6726
Epoch 1, Batch 30, Loss: 0.5487
Epoch 1, Batch 40, Loss: 0.4015
Epoch 1, Batch 50, Loss: 0.3055
Epoch 1, Batch 60, Loss: 0.2939
Epoch 1, Batch 70, Loss: 0.2013
Epoch 1, Batch 80, Loss: 0.2095
Epoch 1, Batch 90, Loss: 0.1939
Epoch 1, Batch 100, Loss: 0.2136
Epoch 1, Batch 110, Loss: 0.1544
Epoch 1, Batch 120, Loss: 0.1439
Epoch 1, Batch 130, Loss: 0.1583
Epoch 1, Batch 140, Loss: 0.1755
Epoch 1, Batch 150, Loss: 0.1557
Epoch 1, Batch 160, Loss: 0.1570
Epoch 1, Batch 170, Loss: 0.1126
Epoch 1, Batch 180, Loss: 0.1351
Epoch 1, Batch 190, Loss: 0.1240
Epoch 1, Batch 200, Loss: 0.1157
Epoch 1, Batch 210, Loss: 0.1137
Epoch 1, Batch 220, Loss: 0.1478
Epoch 1, Batch 230, Loss: 0.1201
Epoch 1, Batch 240, Loss: 0.1270
Epoch 1, Batch 250, Loss: 0.1319
Epoch 1, Batch 260, Loss: 0.1558
Epoch 1, Batch 270, Loss: 0.1434
Epoch 1, Train Loss: 0.2295
Epoch 1, Validation Loss: 0.1679
Epoch 2, Batch 10, Loss: 0.1213
Epoch 2, Batch 20, Loss: 

Traceback (most recent call last):
  File "/home/tim_ivan/idp/util.py", line 110, in model_pipeline
    evaluate_model(model, testloader, device)
  File "/home/tim_ivan/idp/util.py", line 87, in evaluate_model
    print('TP:', tp.item(), 'FP:', fp.item(), 'FN:', fn.item(), 'TN:', tn.item())
RuntimeError: a Tensor with 2 elements cannot be converted to Scalar


epoch,▁▂▃▃▄▅▆▆▇█
train/loss,█▃▂▂▂▁▁▁▁▁
validation/loss,█▇▂▄▄▂▂▁▁▃
epoch,10
train/loss,0.08791
validation/loss,0.1129


RuntimeError: a Tensor with 2 elements cannot be converted to Scalar

In [2]:
import segmentation_models_pytorch as smp
import torch

model = smp.Unet('resnext50_32x4d', in_channels=1, classes=1)
model.load_state_dict(torch.load('./models/Unet_resnext50_32x4d_DiceLoss/Unet-resnet34-DiceLoss_epoch10.pth'))

/home/tim_ivan/idp/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

# Evaluation

In [3]:
from util import evaluate_model
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model.to(device)
evaluate_model(model, testloader, device, with_wandb=False)

Precision: 0.9240, Recall: 0.8203, F1 Score: 0.8663, Dice Score: 0.8663, Accuracy: 0.9833
